In [12]:
!git clone https://github.com/pnpnpn/dna2vec

Cloning into 'dna2vec'...
remote: Enumerating objects: 226, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 226 (delta 2), reused 0 (delta 0), pack-reused 220 (from 1)
Receiving objects: 100% (226/226), 32.73 MiB | 12.76 MiB/s, done.
Resolving deltas: 100% (89/89), done.


In [13]:
cd /content/dna2vec

/content/dna2vec


In [14]:
!pip3 install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.7/81.7 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 23.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.1/178.1 kB 19.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 50.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 8.9 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: T

In [16]:
!pip install logbook

In [17]:
!pip install configargparse

In [18]:
!pip install Bio

In [19]:
!pip install arrow

In [20]:
!pip install gensim

In [21]:
!pip install clock

In [30]:
!pip install gensim==3.8.3


  Using cached gensim-3.8.3.tar.gz (23.4 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for gensim
  Running setup.py clean for gensim
Failed to build gensim
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (gensim)


In [73]:
from pathlib import Path

input_fasta = 'B_Enhancer.txt'
output_dir = Path('/content/dna2vec/inputs/hg38')
output_dir.mkdir(exist_ok=True)

with open(input_fasta, 'r') as infile:
    count = 1
    header, sequence = None, []

    for line in infile:
        line = line.strip()
        if line.startswith('>'):
            if header:
                with open(output_dir / f'enhancer{count}.fa', 'w') as out:
                    out.write(f'{header}\n{"".join(sequence)}\n')
                count += 1
            header = line
            sequence = []
        else:
            sequence.append(line)

    if header:
        with open(output_dir / f'enhancer{count}.fa', 'w') as out:
            out.write(f'{header}\n{"".join(sequence)}\n')


In [74]:
from pathlib import Path

input_fasta = 'B_NonEnhancer.txt'
output_dir = Path('/content/dna2vec/inputs/hg38')
output_dir.mkdir(exist_ok=True)

with open(input_fasta, 'r') as infile:
    count = 1
    header, sequence = None, []

    for line in infile:
        line = line.strip()
        if line.startswith('>'):
            if header:
                with open(output_dir / f'B_NonEnhancer{count}.fa', 'w') as out:
                    out.write(f'{header}\n{"".join(sequence)}\n')
                count += 1
            header = line
            sequence = []
        else:
            sequence.append(line)

    if header:
        with open(output_dir / f'enhancer{count}.fa', 'w') as out:
            out.write(f'{header}\n{"".join(sequence)}\n')


In [31]:
from __future__ import print_function

import logbook
import tempfile
import numpy as np

from gensim.models import word2vec
from gensim import matutils

class SingleKModel:
    def __init__(self, model):
        self.model = model
        self.vocab_lst = sorted(model.vocab.keys())


In [ ]:
# from gensim.models import KeyedVectors
# import logbook

# class MultiKModel:
#     def __init__(self, filepath):
#         self.aggregate = KeyedVectors.load_word2vec_format(filepath, binary=False)
#         self.logger = logbook.Logger(self.__class__.__name__)

    # def cosine_distance(self, seq1, seq2):
    #     # Example: avg vector for all valid kmers in the sequence
    #     def embed_sequence(seq, k=6):
    #         vectors = []
    #         for i in range(len(seq) - k + 1):
    #             kmer = seq[i:i+k].upper()
    #             if kmer in self.aggregate:
    #                 vectors.append(self.aggregate[kmer])
    #         if not vectors:
    #             return None
    #         return sum(vectors) / len(vectors)

    #     vec1 = embed_sequence(seq1)
    #     vec2 = embed_sequence(seq2)
    #     if vec1 is None or vec2 is None:
    #         return float('nan')
    #     return self.aggregate.cosine_similarities(vec1, [vec2])[0]




from gensim.models import KeyedVectors

class MultiKModel:
    def __init__(self, filepath):
        self.aggregate = KeyedVectors.load_word2vec_format(filepath, binary=False)
        self.logger = logbook.Logger(self.__class__.__name__)

        vocab_lens = [len(vocab) for vocab in self.aggregate.key_to_index.keys()]
        self.k_low = min(vocab_lens)
        self.k_high = max(vocab_lens)
        self.vec_dim = self.aggregate.vector_size

        self.data = {}
        for k in range(self.k_low, self.k_high + 1):
            self.data[k] = self.separate_out_model(k)

    def cosine_distance(self, seq1, seq2):
        def embed_sequence(seq, k=6):
            vectors = []
            for i in range(len(seq) - k + 1):
                kmer = seq[i:i+k].upper()
                if kmer in self.aggregate:
                    vectors.append(self.aggregate[kmer])
            if not vectors:
                return None
            return sum(vectors) / len(vectors)

        vec1 = embed_sequence(seq1)
        vec2 = embed_sequence(seq2)
        if vec1 is None or vec2 is None:
            return float('nan')
        return self.aggregate.cosine_similarities(vec1, [vec2])[0]


    def vector(self, vocab):
        k_len = len(vocab)
        if k_len not in self.data:
            raise KeyError(f"No model for k={k_len}")
        return self.data[k_len].model[vocab]

    def separate_out_model(self, k_len):
        vocabs = [vocab for vocab in self.aggregate.key_to_index.keys() if len(vocab) == k_len]
        if len(vocabs) != 4 ** k_len:
            self.logger.warn('Missing {}-mers: {} / {}'.format(k_len, len(vocabs), 4 ** k_len))

        header_str = '{} {}'.format(len(vocabs), self.vec_dim)
        with tempfile.NamedTemporaryFile(mode='w') as fptr:
            print(header_str, file=fptr)
            for vocab in vocabs:
                vec_str = ' '.join("%f" % val for val in self.aggregate[vocab])
                print('{} {}'.format(vocab, vec_str), file=fptr)
            fptr.flush()
            return SingleKModel(KeyedVectors.load_word2vec_format(fptr.name, binary=False))


In [ ]:
import sys
sys.path.append('/content/dna2vec/dna2vec')

In [75]:
!python3 ./scripts/train_dna2vec.py -c configs/hg38-20161219-0153.yml

Streaming output truncated to the last 5000 lines.
[2025-10-27 17:38:45.468010] INFO: SeqGenerator: Whole fasta seqlen: 200
[2025-10-27 17:38:45.468635] INFO: SeqGenerator: Opened file: inputs/hg38/B_NonEnhancer1452.fa
[2025-10-27 17:38:45.468734] INFO: SeqGenerator: Memory usage: 0.726536 MB
[2025-10-27 17:38:45.468778] INFO: SeqGenerator: Current epoch: 2 / 2
[2025-10-27 17:38:45.468864] INFO: SeqGenerator: Whole fasta seqlen: 200
[2025-10-27 17:38:45.469436] INFO: SeqGenerator: Opened file: inputs/hg38/enhancer111.fa
[2025-10-27 17:38:45.469517] INFO: SeqGenerator: Memory usage: 0.726536 MB
[2025-10-27 17:38:45.469564] INFO: SeqGenerator: Current epoch: 2 / 2
[2025-10-27 17:38:45.469646] INFO: SeqGenerator: Whole fasta seqlen: 200
[2025-10-27 17:38:45.470240] INFO: SeqGenerator: Opened file: inputs/hg38/B_NonEnhancer512.fa
[2025-10-27 17:38:45.470301] INFO: SeqGenerator: Memory usage: 0.726536 MB
[2025-10-27 17:38:45.470346] INFO: SeqGenerator: Current epoch: 2 / 2
[2025-10-27 17:38